In [1]:
'''

    2차 TEAM PROJECT

    팀 이름 : 찾아조

    팀원 : 박세영(팀장), 김광휘, 박나현, 서경보

    주제 : 공통 키워드에 대한 언론사별 기사 비교
        - 웹페이지를 구축하여 사용자에게 입력받은 키워드로 6개 언론사의 기사를 검색
        - 검색결과를 웹페이지, 워드파일, PDF파일로 출력하며
        - 회원으로 등록된 사용자의 정보가 있을 경우 검색결과에 대한 보고서를 이메일 발송

'''


''' 
    모듈 import
'''
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import requests
import re
import urllib.request as req
from urllib.parse import quote  # urllib의 parse에서부터 quote 기능을 가져온다. DOM


'''
--------------------------------------------------------------------------------
    사용자에게 키워드 입력받기 
    
    coded by : 김광휘
--------------------------------------------------------------------------------
'''

keyword = str(input('KEYWORD를 입력하세요 : '))
# keyword = '코로나 백신'
xkwd = quote(keyword)

'''
-----END----------END----------END----------END----------END----------END-----
'''

'''
--------------------------------------------------------------------------------
    언론사별 키워드 검색결과 추출 
--------------------------------------------------------------------------------
'''

def munhwa(keyword):
    '''
        문화일보
        coded by : 김광휘
    '''
    url = 'http://mhsearch.munhwa.com/search.php?query='+xkwd
    html = urlopen(url)
    # print(url)
    soup = BeautifulSoup(html, 'lxml')

    # title 추출
    title = soup.select_one('ul > li > div > a > strong').text
    # 날짜 추출
    date = soup.select_one('div > ul > li > div > a > span.data').text.strip()[-10:].replace("-",".")
    # link 추출
    link = soup.select_one('div > ul > li > div > a').attrs['href']
    # 요약 추출
    summary = soup.select_one('div > div > ul > li > div > a > span').text[:200]
    return '문화일보',title, date, link, summary
    # print(title)
    # print(date)
    # print(link)
    # print(summary)

def hankyoreh(keyword):
    '''
        한겨례 신문
        coded by 김광휘
    '''
    url = 'http://search.hani.co.kr/Search?command=query&keyword='+xkwd
    html = urlopen(url)
    # print(url)
    soup = BeautifulSoup(html, 'lxml')

    # title 추출
    lk = soup.select_one('li > dl > dt > a')
    title = lk.text
    # link 추출
    link = "http:" + str(lk.attrs['href'])
    # 날짜 추출
    date = soup.select_one('dl > dd.date > dl > dd').text[:10]
    # 요약 추출
    summary = soup.select_one('ul.search-result-list > li > dl > dd.detail').text.strip()[:200]
    return '한겨례신문',title, date, link, summary
    # print(title)
    # print(link)
    # print(dt)
    # print(smr)

def khan(keyword):
    '''
        경향신문
        coded by : 박세영
    '''
    url = 'http://search.khan.co.kr/search.html?stb=khan&q=' + xkwd

    req = requests.get(url + '&pg=' + '1' + '&sort=2')
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    n_title = soup.select_one('dl.phArtc > dt > a').text
    n_date = soup.select_one('dl.phArtc > dt > span.date').text[1:13]
    a_text = soup.select_one('dl.phArtc > dd.txt').text
    n_url = soup.select_one('dl.phArtc > dt > a').get('href')
    # print(n_title)
    # print(n_date)
    # print(a_text)
    # print(n_url)
    return '경향신문', n_title, n_date, n_url, a_text

def ohmy(keyword):
    '''
        오마이뉴스
        coded by : 박세영
    '''
    url= 'http://www.ohmynews.com/NWS_Web/Search/s_news.aspx?keyword=' + xkwd
    # url2 = '&srh_area=1&srhdate=1&s_year=2021&s_month=1&s_day=30&e_year=2021&e_month=4&e_day=30&section_code=0&area_code=0&form_code=0'

    req = requests.get(url)  # + url2 + '&page=' + '1')
    # req = requests.get(url1 + keyword + url2 + '&page=' + '1')
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    n_title = soup.select_one('div.cont > dl > dt > a').text
    n_date = soup.select_one('div.cont > p.source').text[-10:]
    a_text = soup.select_one('div.cont > dl > dd').text
    n_url = 'http://www.ohmynews.com' + soup.select_one('div.cont > dl > dt > a').get('href')
    # print(n_title)
    # print(n_date)
    # print(a_text)
    # print(n_url)
    return "오마이뉴스", n_title, n_date, n_url, a_text

def DongA(keyword):
    '''
        동아일보
        coded by : 박나현
    '''
    TARGET_URL_BEFORE_PAGE_NUM = "http://news.donga.com/search?p="
    TARGET_URL_BEFORE_KEWORD = '&query='
    TARGET_URL_REST = '&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=3'

    url = TARGET_URL_BEFORE_PAGE_NUM + TARGET_URL_BEFORE_KEWORD + xkwd + TARGET_URL_REST
    res = req.urlopen(url).read()
    soup = BeautifulSoup(res, 'html.parser')

    title = soup.select('div.searchContWrap > div.searchCont > div > div.t > p.tit > a')[0].text
    contents = soup.select('div.searchContWrap > div.searchCont > div > div.t > p.txt > a')[0].text
    link = soup.select('div.searchContWrap > div.searchCont > div > div.p > a')[0].get('href')
    date = soup.select('div.searchContWrap > div.searchCont > div > div.t > p.tit > span')[0].text[:10]
    date = date.replace("-",".")
    return "동아일보", title, date, link, contents

def jungang(keyword):
    '''
        중앙일보
        coded by : 서경보
    '''
    try:
        # keyword = input('키워드를 입력하세요: ')  # 키워드입력
        url = 'https://news.joins.com/search/?keyword=' + xkwd  # 중앙일보검색사이트에 내가 적은 키워드 더하기
        html = urlopen(url)
        bsobject = BeautifulSoup(html, "html.parser")  # 크롤링

        title = bsobject.select('#searchNewsArea > div.bd > ul > li > div > h2 > a')  # 기사제목추출 list로추출
        article_title = title[0].text  # 추출된 기사에서 제목추출
        article_url = title[0].get('href')  # 추출된 기사에서 url추출

        summary = bsobject.select('#searchNewsArea > div.bd > ul > li > div > span.lead')  # 기사요약추출 list추출
        article_summary = summary[0].text  # 기사요약추출에서 요약추출

        date = bsobject.select(
            '#searchNewsArea > div.bd > ul > li > div > span.byline > em')  # 날짜 list추출
        article_date = date[1].text[:10]  # 날짜


        # print('\n기사제목 : {0} \n\n기사내용 : {1}  \n\n기사날짜 : {2} \n\n기사주소 : {3}'.format(article_title, article_summary,
        #                                                                           article_date, article_url))
        return "중앙일보", article_title, article_date, article_url, article_summary

        # 기사가안나올때 에러생겨. 예외처리하는거 ex/ 기사없을떄 => none 출력
    except:
        print('없는 검색 결과 입니다.')


mh = munhwa(keyword)
hkr = hankyoreh(keyword)
khan = khan(keyword)
ohmy = ohmy(keyword)
dnga = DongA(keyword)
jng = jungang(keyword)
'''
-----END----------END----------END----------END----------END----------END-----
'''



'''
--------------------------------------------------------------------------------
    추출 검색결과로 데이터 프레임 작성 

    coded by : 김광휘
--------------------------------------------------------------------------------
'''

n_list = [dnga, jng, mh, hkr, khan, ohmy]
columns = ['언론사', '제목', '날짜', 'URL', '요약']

df = pd.DataFrame(n_list, columns=columns)
#print(df[['제목','URL']])
#print(df.loc[0,'URL'])

def news_article_DongA(keyword):
    '''
       동아일보
    '''
    url_DongA = df.loc[0,'URL']
    resp = requests.get(url_DongA)
    soup = BeautifulSoup(resp.text)

    #뉴스 제목 가져오기
    title = soup.select_one('h1.title')
    news_title = title.get_text()

    #뉴스 본문 가져오기
    article = soup.select_one('div.article_txt')
    news_article = article.get_text()
    return news_title, news_article

def news_article_Jungang(keyword):
    '''
       중앙일보
    '''
    url_Jungang = df.loc[1,'URL']
    #print(url_Jungang)
    resp = requests.get(url_Jungang)
    soup = BeautifulSoup(resp.text)

    #뉴스 제목 가져오기
    title = soup.select_one('h1#article_title')
    news_title = title.get_text()
    #print(news_title)

    #뉴스 본문 가져오기
    article = soup.select_one('div#article_body')
    news_article = article.get_text()
    #print(news_article)
    return news_title, news_article

def news_article_Munhwa(keyword):
    '''
       문화일보
    '''
    url_Munhwa = df.loc[2,'URL']
    #print(url_Munhwa)
    resp = requests.get(url_Munhwa)
    soup = BeautifulSoup(resp.content,'html.parser', from_encoding='utf-8')

    #뉴스 제목 가져오기
    title = soup.select_one('span.title')
    news_title = title.get_text()
    #print(news_title)
    
    #뉴스 본문 가져오기
    article = soup.select_one('div#NewsAdContent')
    news_article = article.get_text()
    #print(news_article)
    return news_title, news_article

def news_article_Hankyoreh(keyword):
    '''
       한겨레
    '''
    url_Hankyoreh = df.loc[3,'URL']
    #print(url_Hankyoreh)
    resp = requests.get(url_Hankyoreh)
    soup = BeautifulSoup(resp.content,'html.parser', from_encoding='utf-8')

    #뉴스 제목 가져오기
    title = soup.select_one('span.title')
    news_title = title.get_text()
    #print(news_title)
    
    #뉴스 본문 가져오기
    article = soup.select_one('div.text')
    news_article = article.get_text()
    #print(news_article)
    return news_title, news_article
 
def news_article_Ohmy(keyword):
    '''
       오마이뉴스
    '''
    url_Ohmy = df.loc[5,'URL']
    #print(url_Ohmy)
    resp = requests.get(url_Ohmy)
    soup = BeautifulSoup(resp.content,'html.parser', from_encoding='utf-8')

    #뉴스 제목 가져오기
    title = soup.select_one('h3.tit_subject')
    news_title = title.get_text()
    #print(news_title)
    
    #뉴스 본문 가져오기
    article = soup.select_one('div.at_contents')
    news_article = article.get_text()
    #print(news_article)
    return news_title, news_article


'''-------------------데이터프레임 만들기--------------------'''

# 동아, 중앙, 문화 데이터프레임 만들기
r_list = [news_article_DongA(keyword), news_article_Jungang(keyword), news_article_Munhwa(keyword)]
r_columns = ['제목', '본문']

r_df = pd.DataFrame(r_list, columns=r_columns)
#print(r_df)

# 한겨례, 경향, 오마이뉴스 데이터 프레임 만들기 (+) 경향 추가 해야함!
b_list = [news_article_Hankyoreh(keyword), news_article_Ohmy(keyword)]
b_columns = ['제목', '본문']

b_df = pd.DataFrame(b_list, columns=b_columns)
#print(b_df)

'''--------------------데이터프레임 txt로 저장하기----데이터프레임으로 바로 하면 기사전문 안 나옴 왜 그런지는몰라..--------------'''

#import pandas as pd

fname = 'search_result_'+ keyword
r_df.to_csv(fname+' r'+'.txt', sep = '\t', index=False)
b_df.to_csv(fname+' b'+'.txt', sep = '\t', index=False)

'''------------------워드 클라우드 ---------------------'''
from konlpy.tag import Okt, Twitter
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

# 글자 색 지정하는 함수 : red, blue 나누었음
def color_func_red(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d}, {:d}%, {:d}%)".format(np.random.randint(4,6),np.random.randint(56,78),np.random.randint(57,95)))

def color_func_blue(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d}, {:d}%, {:d}%)".format(np.random.randint(230,231),np.random.randint(44,48),np.random.randint(48,94)))

# 명사, 2글자 이상만 꺼내오는 함수 지정 : red, blue 나눔
def get_noun_red(news_r):
    
    okt=Okt()
    noun = okt.nouns(news_r)
    for i,v in enumerate(noun):
        if len(v) < 2:
            noun.pop(i)
            
    count = Counter(noun)
    noun_list_r = count.most_common(100)
    
    return noun_list_r

def get_noun_blue(news_b):
    
    okt=Okt()
    noun = okt.nouns(news_b)
    for i,v in enumerate(noun):
        if len(v) < 2:
            noun.pop(i)
            
    count = Counter(noun)
    noun_list_b = count.most_common(100)
    
    return noun_list_b

# red -> 고래 / blue -> 코끼리 각각 시각화,파일 저장 ## 폰트, 사진 자료 필요함 ##
def visualize_red(noun_list_r):
    
    whale_mask = np.array(Image.open("whale.jpg"))
       
    wc = WordCloud(font_path='Recipekorea.ttf',\
              background_color = "white", \
              width = 1000, \
              height=1000, \
              max_words=100, \
              max_font_size=300, \
              mask=whale_mask, \
              color_func = color_func_red)
    
    wc.generate_from_frequencies(dict(noun_list_r))
    wc.to_file(fname+'wordcloud_red'+'.png')
    
def visualize_blue(noun_list_b):
    
    whale_mask = np.array(Image.open("elephant.jpg"))
       
    wc = WordCloud(font_path='Recipekorea.ttf',\
              background_color = "white", \
              width = 1000, \
              height=1000, \
              max_words=100, \
              max_font_size=300, \
              mask=whale_mask, \
              color_func = color_func_blue)
    
    wc.generate_from_frequencies(dict(noun_list_b))
    wc.to_file(fname+'wordcloud_blue'+'.png')

# red, blue 함수 경로 지정
if __name__=='__main__':
    filename = sys.argv[1]
    
    fr= open(fname+' r'+'.txt', 'r', encoding='utf-8')
    news_r = fr.read()
    noun_list_r = get_noun_red(news_r)
    visualize_red(noun_list_r)
    
    fb= open(fname+' b'+'.txt', 'r', encoding='utf-8')
    news_b = fb.read()
    noun_list_b = get_noun_blue(news_b)
    visualize_blue(noun_list_b)


KEYWORD를 입력하세요 : AZ
